<a href="https://colab.research.google.com/github/ydv1412/live_face_detection/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from tensorflow.keras.optimizers import RMSprop
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
train_generator = ImageDataGenerator(rescale = 1/255.0) 

test_generator = ImageDataGenerator(rescale = 1/255.0 , validation_split = 0.5) 

In [ ]:
train_Data = train_generator.flow_from_directory('//content//drive//MyDrive//capstone project //computer vision_face_emotion_recognisation//dataset//train' ,target_size = (224,224), batch_size= 128 , class_mode = 'binary')

Found 10420 images belonging to 7 classes.


In [ ]:
validation_Data = test_generator.flow_from_directory('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/dataset/test' ,target_size = (224,224), batch_size= 128 , class_mode = 'binary' , subset = 'validation')

Found 3587 images belonging to 7 classes.


In [ ]:
test_Data = test_generator.flow_from_directory('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/dataset/test' ,target_size = (224,224), batch_size= 128 , class_mode = 'binary',subset = 'training')

Found 3591 images belonging to 7 classes.


In [ ]:
base_model = VGG16(input_shape = (224,224,3))

553476096/553467096 [==============================] - 5s 0us/step


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
base_input = base_model.layers[0].input
base_output = base_model.layers[-2].output

In [ ]:
model = keras.Model( inputs = base_input , outputs = base_output)

In [ ]:
model.trainable = False

In [ ]:
x = model.output
final_model = keras.layers.Dense(128 , activation = 'relu')(x)
final_model = keras.layers.Dropout(0.5)
final_model = keras.layers.Dense(64 , activation = 'relu')(final_model)
final_model = keras.layers.Dropout(0.5)
final_model = keras.layers.Denses(32 , activation = 'relu')(final_model)
final_model = keras.layers.Dropout(0.5)
final_model = keras.layers.Dense(7 , activation = 'softmax')(final_model)


TypeError: ignored

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg_model.layers[:15]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [ ]:
x = vgg_model.output
x = keras.layers.Flatten()(x) # Flatten dimensions to for use in FC layers
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(8, activation='softmax')(x) # Softmax for multiclass
transfer_model = keras.Model(inputs=vgg_model.input, outputs=x)

In [ ]:
transfer_model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [ ]:
transfer_model.fit(train_Data, epochs=25, validation_data=validation_Data)

Epoch 1/25
82/82 [==============================] - 3256s 40s/step - loss: 1.3378 - accuracy: 0.3748 - val_loss: 3.9122 - val_accuracy: 0.1427
Epoch 2/25
82/82 [==============================] - 123s 2s/step - loss: 1.1911 - accuracy: 0.3924 - val_loss: 4.8079 - val_accuracy: 0.1399
Epoch 3/25
82/82 [==============================] - 123s 2s/step - loss: 1.1839 - accuracy: 0.3866 - val_loss: 4.3242 - val_accuracy: 0.1408
Epoch 4/25
82/82 [==============================] - 123s 1s/step - loss: 1.1451 - accuracy: 0.4119 - val_loss: 5.8752 - val_accuracy: 0.2975
Epoch 5/25
82/82 [==============================] - 123s 1s/step - loss: 0.9661 - accuracy: 0.5546 - val_loss: 6.9036 - val_accuracy: 0.3195
Epoch 6/25
82/82 [==============================] - 123s 1s/step - loss: 0.8642 - accuracy: 0.6153 - val_loss: 7.3522 - val_accuracy: 0.3306
Epoch 7/25
82/82 [==============================] - 123s 1s/step - loss: 0.7860 - accuracy: 0.6616 - val_loss: 7.0507 - val_accuracy: 0.3354
Epoch 8/25


In [ ]:
vgg_model2 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x1 = vgg_model2.output
x1 = keras.layers.Flatten()(x1) # Flatten dimensions to for use in FC layers
x1 = keras.layers.Dense(512, activation='relu')(x1)
x1 = keras.layers.Dropout(0.5)(x1) # Dropout layer to reduce overfitting
x1 = keras.layers.Dense(256, activation='relu')(x1)
x1 = keras.layers.Dense(7, activation='softmax')(x1) # Softmax for multiclass
transfer_model2 = keras.Model(inputs=vgg_model2.input, outputs=x1)
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
transfer_model2.compile(loss = 'sparse_categorical_crossentropy' , optimizer = opt , metrics = ['accuracy'])

In [ ]:
history = transfer_model2.fit(train_Data, epochs=25, validation_data=validation_Data)

Epoch 1/25
82/82 [==============================] - 1989s 24s/step - loss: 1.2580 - accuracy: 0.3939 - val_loss: 3.6118 - val_accuracy: 0.1915
Epoch 2/25
82/82 [==============================] - 335s 4s/step - loss: 0.9915 - accuracy: 0.5490 - val_loss: 4.1372 - val_accuracy: 0.3298
Epoch 3/25
82/82 [==============================] - 335s 4s/step - loss: 0.8296 - accuracy: 0.6459 - val_loss: 4.1035 - val_accuracy: 0.3872
Epoch 4/25
82/82 [==============================] - 335s 4s/step - loss: 0.7273 - accuracy: 0.6951 - val_loss: 4.0476 - val_accuracy: 0.4001
Epoch 5/25
82/82 [==============================] - 335s 4s/step - loss: 0.6586 - accuracy: 0.7295 - val_loss: 4.1506 - val_accuracy: 0.4003
Epoch 6/25
82/82 [==============================] - 335s 4s/step - loss: 0.6022 - accuracy: 0.7583 - val_loss: 4.4438 - val_accuracy: 0.3995
Epoch 7/25
82/82 [==============================] - 335s 4s/step - loss: 0.5153 - accuracy: 0.7937 - val_loss: 4.5638 - val_accuracy: 0.4090
Epoch 8/25


In [ ]:
transfer_model2.save_weights('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/vgg_trainable.h5')

In [ ]:
vgg_model2 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x2 = vgg_model2.output
x2 = keras.layers.Flatten()(x2)
x2 = keras.layers.Dense(7, activation='softmax')(x2) # Softmax for multiclass
transfer_model3 = keras.Model(inputs=vgg_model2.input, outputs=x2)
transfer_model3.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [ ]:
history2 = transfer_model3.fit(train_Data, epochs=25, validation_data=validation_Data)

Epoch 1/25


InvalidArgumentError: ignored